In [1]:
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 16 # number of independent sequences processed in parallel?
block_size = 32 # maximum context length for predictions
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [3]:
torch.manual_seed(1337)

In [4]:
# Download Jambi language dataset
!wget https://raw.githubusercontent.com/Mufaddhal/Projects/refs/heads/main/GPT-with-Jambi-lang-dev/Jambi-Language-Dataset.txt

--2025-05-28 13:20:05--  https://raw.githubusercontent.com/Mufaddhal/Projects/refs/heads/main/GPT-with-Jambi-lang-dev/Jambi-Language-Dataset.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 789276 (771K) [text/plain]
Saving to: ‘Jambi-Language-Dataset.txt’

Jambi-Language-Data 100%[===================>] 770.78K  --.-KB/s    in 0.06s   

2025-05-28 13:20:06 (13.6 MB/s) - ‘Jambi-Language-Dataset.txt’ saved [789276/789276]



In [5]:
with open('Jambi-Language-Dataset.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
# Length of the dataset in characters
print("length of dataset in characters: ", len(text))

length of dataset in characters:  779120


In [7]:
# The first 1000 characters
print(text[:1000])

Ngapo Ado Banyak Angso? Mengapa Ada Banyak Angsa?

Fenny baru be tibo di Jambi. Diok ningok pemandangan kota dari jenelo mubil.
“Wah, di sano ado patung angso puti!” Fenny baru saja tiba di Jambi. Dia melihat
pemandangan kota dari jendela mobil. “Wah, di sana ada patung angsa putih!” Di
bandara ado patung angso. Di gapuranyo jugo ado. Malah di kolam dekat bandara,
ado angso berenang. Wah, di mano-mano ado angso! Di bandara ada patung angsa. Di
gapuranya juga ada. Bahkan di kolam dekat bandara, ada angsa berenang. Wah, di
mana-mana ada angsa!  Fenny baru be tibo di Jambi. Diok ningok pemandangan
kota dari jenelo mubil. “Wah, di sano ado patung angso puti!” Fenny baru saja
tiba di Jambi. Dia melihat pemandangan kota dari jendela mobil. “Wah, di sana
ada patung angsa putih!” Di bandara ado patung angso. Di gapuranyo jugo ado.
Malah di kolam dekat bandara, ado angso berenang. Wah, di mano-mano ado angso!
Di bandara ada patung angsa. Di gapuranya juga ada. Bahkan di kolam dekat
bandara, ada

In [8]:
# Vocab size
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"%'+,-./0123456789?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz“”
78


In [9]:
# A mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("Jambi"))
print(decode(encode("Jambi")))

[31, 50, 62, 51, 58]
Jambi


In [10]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([779120]) torch.int64
tensor([35, 56, 50, 65, 64,  1, 22, 53, 64,  1, 23, 50, 63, 74, 50, 60,  1, 22,
        63, 56, 68, 64, 21,  1, 34, 54, 63, 56, 50, 65, 50,  1, 22, 53, 50,  1,
        23, 50, 63, 74, 50, 60,  1, 22, 63, 56, 68, 50, 21,  0,  0, 27, 54, 63,
        63, 74,  1, 51, 50, 67, 70,  1, 51, 54,  1, 69, 58, 51, 64,  1, 53, 58,
         1, 31, 50, 62, 51, 58,  9,  1, 25, 58, 64, 60,  1, 63, 58, 63, 56, 64,
        60,  1, 65, 54, 62, 50, 63, 53, 50, 63, 56, 50, 63,  1, 60, 64, 69, 50,
         1, 53, 50, 67, 58,  1, 59, 54, 63, 54, 61, 64,  1, 62, 70, 51, 58, 61,
         9,  0, 76, 44, 50, 57,  7,  1, 53, 58,  1, 68, 50, 63, 64,  1, 50, 53,
        64,  1, 65, 50, 69, 70, 63, 56,  1, 50, 63, 56, 68, 64,  1, 65, 70, 69,
        58,  2, 77,  1, 27, 54, 63, 63, 74,  1, 51, 50, 67, 70,  1, 68, 50, 59,
        50,  1, 69, 58, 51, 50,  1, 53, 58,  1, 31, 50, 62, 51, 58,  9,  1, 25,
        58, 50,  1, 62, 54, 61, 58, 57, 50, 69,  0, 65, 54, 62, 50, 63, 53, 50,
       

In [11]:
n = int(0.9*len(data)) # 90% train set, dan 10% val set
train_data = data[:n]
val_data = data[n:]

In [12]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [13]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [14]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [15]:
model = BigramLanguageModel()
m = model.to(device)
# Number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.211406 M parameters


In [16]:
# Create Adam Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [17]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.5305, val loss 4.5376
step 100: train loss 2.4499, val loss 2.4616
step 200: train loss 2.3713, val loss 2.3704
step 300: train loss 2.2991, val loss 2.3120
step 400: train loss 2.2552, val loss 2.2587
step 500: train loss 2.2224, val loss 2.2241
step 600: train loss 2.1866, val loss 2.1871
step 700: train loss 2.1583, val loss 2.1657
step 800: train loss 2.1240, val loss 2.1298
step 900: train loss 2.0997, val loss 2.1186
step 1000: train loss 2.0688, val loss 2.0799
step 1100: train loss 2.0510, val loss 2.0573
step 1200: train loss 2.0236, val loss 2.0395
step 1300: train loss 1.9889, val loss 2.0082
step 1400: train loss 1.9618, val loss 1.9789
step 1500: train loss 1.9477, val loss 1.9746
step 1600: train loss 1.9210, val loss 1.9491
step 1700: train loss 1.9005, val loss 1.9213
step 1800: train loss 1.8873, val loss 1.9122
step 1900: train loss 1.8758, val loss 1.9015
step 2000: train loss 1.8622, val loss 1.8811
step 2100: train loss 1.8413, val loss 1.8788


In [18]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


melakukan evee dari pengela
sudah di bisa jangan mau Anda Peritangan pas lanjut tiha-temukan pukul ke eenal di Kemasa Balat 2 di Bitatasi bulan dalam
Pak dari tua dan di ilai Saterasi kan palta dapat juta dan penungungan Buar buu jelan ik bnak acau nya, berita tuh. Itu kayu Sudar ya sana Tanti Haara mau aku belum destaen jalan Salidara dan ini oleh bul-can. Satu poram itu ditega penerima jaco cepat Kenaui menjur misalkan mau kalau yang jalan melakukan pelaku
I uara apdulinya Pakinggala mau ada No ini kita tinggi dengan Pak RT I diri
kapor yaita hamir tapi ada hahus Ren 252 E5 ojs jam
mau gitu entu ee biasan kacet nih kan Kami walau lokasi dengan di kami kita Dekaran dulu pakai biasksanya PPuk Durtuk ahai menyelama jatara kendaraan bisa berekapal sabu. Tapas guna melikiaf iturusnya jambis duan tuh kan terbukan kegi semang tidak
pasla dia koretaka nih KMura kepala Kayam bawan berhasilah itu baik proses segeturusnya masih api dibeldukan bagik ada derah Dan barang Tunghur
mereka bilana du